## 1. Introduction
We will reimplement the methodology of the paper in Python.

## 2. Preliminary Concepts

Initially, we will recreate the basic variables defined in the paper. To make calculations easier, we will use NaNs instead of zeros if a movie is not rated by a user.

In [47]:
import numpy as np

m = 6040 # users
n = 3952 # movies
Rnan = np.full((m, n), np.nan) # Ratings matrix with nans instead of 0s
# R_hat = np.zeros((m,n), dtype = np.float) # Predicted rating matrix

We read the data from the ratings file.

In [48]:
import io
# Read the data into the rating matrix
with open('ml-1m/ratings.dat', 'r') as fp:
    for line in iter(fp.readline, ''):
        l = line.split('::')
        Rnan[int(l[0])-1,int(l[1])-1] = int(l[2])
  

We continue defining functions as per the paper. $O_u$ is the item (movie) set of the user $u_i$ and $U_o$ is the user set of the item (movie) $o_j$.

In [49]:
def O_u(u_i): # item set of user u_i
    return np.nonzero(1-np.isnan(Rnan[u_i,:]))
def U_o(o_j): # user set of item o_j
    return np.nonzero(1-np.isnan(Rnan[:,o_j]))

print(O_u(0))
print(U_o(0))

(array([   0,   47,  149,  259,  526,  530,  587,  593,  594,  607,  660,
        719,  744,  782,  913,  918,  937, 1021, 1027, 1028, 1034, 1096,
       1192, 1196, 1206, 1245, 1269, 1286, 1544, 1565, 1720, 1835, 1906,
       1960, 1961, 2017, 2027, 2293, 2320, 2339, 2354, 2397, 2686, 2691,
       2761, 2790, 2796, 2803, 2917, 3104, 3113, 3185, 3407]),)
(array([   0,    5,    7, ..., 6031, 6034, 6039]),)


In [53]:
r_bar_v = np.nanmean(Rnan, axis = 1) # mean ratings of user u_i
r_bar = np.nanmean(Rnan) # global mean rating
print('Average rating:',r_bar)
print('Average rating for user 0:',r_bar_v[0])

Average rating: 3.58156445303
Average rating for user 0: 4.18867924528


We define the prediction function.

In [56]:
def calc_r_hat(u_t, o_j, c_t):
    """
    u_t -> target user
    o_j -> target movie
    c_t -> similarity vector of t to all users
    """
    U_oj = U_o(o_j)
    return r_bar_v[u_t] - r_bar + (np.sum(c_t[U_oj]*Rnan[U_oj, o_j]) / np.sum(c_t[U_oj]))

print('Rating of user 0 on movie 0:', Rnan[0,0])
print('Estimated rating (using uniform user similarity):', calc_r_hat(0,0, np.ones(m)))

Rating of user 0 on movie 0: 5.0
Estimated rating (using uniform user similarity): 4.75396120535


## 3. Random Walk

Instead of defining a probability function from user $u_i$ to movie $o_j$, we calculate the probabilities beforehand and store them in a matrix.

In [57]:
P_uo = (1 - np.isnan(Rnan)) / np.sum(1 - np.isnan(Rnan), axis = 1).reshape(R.shape[0],1) # Type 1 walk, user to movie
print(P_uo)

[[ 0.01886792  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.00293255  0.          0.         ...,  0.          0.          0.        ]]


We also define the rating similarity matrix for the user $u_i$. The computed ratings are only numbers if $u_i$ and $u_k$ both have a rating for that item, where $k\in [0..m]$.

In [58]:
MAXSCORE = 5

def sim(u_i): # similarity matrix from u_k to o_j, given u_i
    return MAXSCORE - np.absolute(Rnan[u_i,:] - Rnan)

sim(0)

array([[  5.,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [  3.,  nan,  nan, ...,  nan,  nan,  nan]])

Using the rating similarity matrix, we can quickly compute the total similarity score for each item $o_j$ over all users, given $u_i$. By not including NaNs, we are calculating the denominators of the Type 2 probabilities.

In [59]:
def sum_sim(u_i): # Sum of similarities for any o_j, give u_i
    return np.nansum(sim(u_i), axis = 0)

sum_sim(0)

array([ 8613.,     0.,     0., ...,     0.,     0.,     0.])

We can now define the probability function from item (movie) $o_j$ to user $u_k$, given the previous transition was from $u_i$ to $o_j$. Again, we calculate a transition probability matrix to lessen the number of computations. Note that we actually return the transpose of the transition probability matrix, to ease further calculations.

In [60]:
def P_ou(u_i):
    """
    Transition probability matrix from movie to user, given
    a base user u_i. Note that axis 0 is still the user and
    the axis 1 is the movie.
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        s = sim(u_i)
        return s / np.nansum(s, axis = 0)

P_ou(0)

array([[ 0.00058052,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ..., 
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 0.00034831,         nan,         nan, ...,         nan,
                nan,         nan]])

We then calculate the transition probability from a user $u_i$ to any other user $u_k$.

In [61]:
def p(u_i):
    """ Transition probability from user u_i to each user. """
    return np.nansum(P_uo[u_i] * P_ou(u_i), axis = 1)
p(0)

array([  2.08975477e-03,   1.39117946e-04,   5.76297717e-05, ...,
         0.00000000e+00,   4.21630861e-04,   2.39080539e-04])

The calculated transition probabilities are stacked on top of each other to build the transition probability matrix $P$.

In [77]:
def construct_P(m):
    l = [p(u_i) for u_i in range(m)]
    return np.vstack(l)

P = construct_P(m)
print(P)

[[  2.08975477e-03   1.39117946e-04   5.76297717e-05 ...,   0.00000000e+00
    4.21630861e-04   2.39080539e-04]
 [  5.75640788e-05   2.37030982e-03   4.78522937e-05 ...,   1.16093427e-05
    7.43764959e-05   4.96480069e-04]
 [  5.94312010e-05   1.18340680e-04   1.69691406e-03 ...,   3.78222708e-05
    8.11348543e-05   2.36556977e-04]
 ..., 
 [  0.00000000e+00   7.61002797e-05   9.69507318e-05 ...,   2.28147300e-03
    6.75691406e-04   9.77292459e-04]
 [  1.71943645e-04   8.10291189e-05   3.25818579e-05 ...,   1.08172873e-04
    3.43334096e-03   4.01288154e-04]
 [  3.66031493e-05   1.90460161e-04   3.62252940e-05 ...,   6.26079966e-05
    1.42890116e-04   4.95235564e-03]]


Since constructing this matrix takes time, we save it on the local machine, so that we don't have to calculate it again.

In [78]:
np.save('P.npy', P)

In [62]:
P = np.load('P.npy')
print(P)

[[  2.08975477e-03   1.39117946e-04   5.76297717e-05 ...,   0.00000000e+00
    4.21630861e-04   2.39080539e-04]
 [  5.75640788e-05   2.37030982e-03   4.78522937e-05 ...,   1.16093427e-05
    7.43764959e-05   4.96480069e-04]
 [  5.94312010e-05   1.18340680e-04   1.69691406e-03 ...,   3.78222708e-05
    8.11348543e-05   2.36556977e-04]
 ..., 
 [  0.00000000e+00   7.61002797e-05   9.69507318e-05 ...,   2.28147300e-03
    6.75691406e-04   9.77292459e-04]
 [  1.71943645e-04   8.10291189e-05   3.25818579e-05 ...,   1.08172873e-04
    3.43334096e-03   4.01288154e-04]
 [  3.66031493e-05   1.90460161e-04   3.62252940e-05 ...,   6.26079966e-05
    1.42890116e-04   4.95235564e-03]]


## 4. Sampling Algorithm

We create a random test set of 10 numbers for our initial walk.

In [65]:
size_ts = 10
ts = np.random.randint(m, size=(size_ts,))
ts

array([ 802, 2890, 3037,  737, 1092, 1768, 4586, 3049, 2162, 4657])